In [1]:
!nvidia-smi

Sat Jul 19 13:50:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets transformers

In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [4]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-hi'

In [5]:
raw_dataset = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [12]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [6]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [7]:
raw_dataset['train'][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

In [8]:
tokenizer  = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
tokenizer('Accerciser Accessibility Explorer')

{'input_ids': [32643, 28541, 36253, 0], 'attention_mask': [1, 1, 1, 1]}

In [10]:
with tokenizer.as_target_tokenizer():
  print(tokenizer('एक्सेर्साइसर पहुंचनीयता अन्वेषक'))

# here i am using the tokenizer to tokenize the target language so we are this using this type of function

{'input_ids': [26618, 16155, 346, 33383, 0], 'attention_mask': [1, 1, 1, 1, 1]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
train_subset = raw_dataset["train"].select(range(20000))
val_dataset = raw_dataset["validation"]  # using full validation set

In [12]:
max_length = 128
max_target_length = 128

source_lang = 'en'
target_lang = 'hi'

def preprocess_function(examples):
  inputs = [ex[source_lang] for ex in examples['translation']]
  targets = [ex[target_lang] for ex in examples['translation']]
  model_inputs = tokenizer(inputs,max_length=max_length,truncation=True)

  # to tokenize the target words
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets,max_length=max_target_length,truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [13]:
preprocess_function(raw_dataset['train'][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [14]:
tokenized_train = train_subset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

In [15]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [16]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_model = 2

In [17]:
data_collector = DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors='tf')
# here basically it takes the data batch by batch

In [18]:
generation_data_collactor = DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors='tf',pad_to_multiple_of=128)

In [19]:

# Prepare the tf.data.Dataset
tf_train_dataset = tokenized_train.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collector
)

# For validation
tf_val_dataset = tokenized_val.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collector
)

In [20]:
optimizer = AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [21]:
model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=2
)

Epoch 1/2
1250/1250 [==============================] - 271s 182ms/step - loss: 0.6351 - val_loss: 4.5142
Epoch 2/2
1250/1250 [==============================] - 237s 190ms/step - loss: 0.2713 - val_loss: 4.6418


In [22]:
model.save_pretrained("english_to_hindi_model")
tokenizer.save_pretrained("english_to_hindi_model")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:407: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


('english_to_hindi_model/tokenizer_config.json',
 'english_to_hindi_model/special_tokens_map.json',
 'english_to_hindi_model/vocab.json',
 'english_to_hindi_model/source.spm',
 'english_to_hindi_model/target.spm',
 'english_to_hindi_model/added_tokens.json')

In [24]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/english_to_hindi_model")
tokenizer = AutoTokenizer.from_pretrained("/content/english_to_hindi_model")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/english_to_hindi_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [25]:
def translate_en_to_hi(sentence, max_length=128):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="tf", padding=True, truncation=True, max_length=max_length)

    # Generate translation IDs
    output_ids = model.generate(

        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )

    # Decode the output to Hindi text
    translated_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return translated_sentence

In [26]:
input_text = "tomorrow can we push it to production sir ?"
tokenized = tokenizer([input_text],return_tensors='np')
out = model.generate(**tokenized,max_length = max_length)
print(out)

tf.Tensor(
[[61949  3093    75   508  2143    24 13768    91     6    39 18229   245
    175    28    22     0]], shape=(1, 16), dtype=int32)


In [27]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True))

कल हम इसे सर का उत्पादन करने के लिए धक्का दे सकते हैं?


In [29]:
import gradio as gr
from transformers import MarianTokenizer, TFMarianMTModel

# Load model and tokenizer
model_name = "/content/english_to_hindi_model"  # change if you're using a custom fine-tuned model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = TFMarianMTModel.from_pretrained(model_name)

# Set a reasonable max length
max_length = 300

# Define prediction function
def translate_to_hindi(input_text):
    # Tokenize input
    tokenized = tokenizer([input_text], return_tensors='tf')  # tf if you're using TensorFlow model
    # Generate translation
    translated = model.generate(**tokenized, max_length=max_length)
    # Decode
    with tokenizer.as_target_tokenizer():
        hindi_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return hindi_text

# Gradio interface
gr.Interface(
    fn=translate_to_hindi,
    inputs=gr.Textbox(lines=2, placeholder="Enter English sentence..."),
    outputs="text",
    title="English to Hindi Translator",
    description="Enter English text and get the Hindi translation"
).launch()

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/english_to_hindi_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6caa4a3714c57b4cc0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
